In [ ]:
# Install llmcompressor Tool
!pip install llmcompressor==0.5.2 -q

In [ ]:
#Validate LLM Compressor is installed
!pip list | grep llmcompressor

In [ ]:
# Run Python script to quantise the model

from quantiser import quantise_model

# You can view this code by opening the Python file from the File Explorer. 
quantise_model("TinyLlama/TinyLlama-1.1B-Chat-v1.0", scheme="W4A16")

In [ ]:
# Review contents of the file 
!ls -lsah TinyLlama/TinyLlama-1.1B-Chat-v1.0-W4A16

In [ ]:
# Evaluate accuracy of the model. We will be using lm-eval script in vLLM.
!pip install vllm==0.9.1 lm-eval==0.4.7 -q

In [ ]:
# Evaluate unquantized model
# !lm-eval --model vllm --model_args pretrained=TinyLlama/TinyLlama-1.1B-Chat-v1.0,tensor_parallel_size=1 \
# --limit 250 --tasks hellaswag --num_fewshot 5 --batch_size 5

import os 
os.environ["HF_TOKEN"] = "token"

!time lm-eval \
  --model vllm \
  --model_args pretrained=TinyLlama/TinyLlama-1.1B-Chat-v1.0,tensor_parallel_size=1,gpu_memory_utilization=0.8 \
  --limit 250 \
  --tasks hellaswag \
  --num_fewshot 5 \
  --batch_size 5

In [ ]:
# Now we run the same command against the post-quantized model.
# Note: You will need specify the Tokenizer explicitly.

!time lm-eval \
  --model vllm \
  --model_args pretrained=TinyLlama/TinyLlama-1.1B-Chat-v1.0-W4A16,tokenizer=TinyLlama/TinyLlama-1.1B-Chat-v1.0,tensor_parallel_size=1,gpu_memory_utilization=0.8 \
  --limit 250 \
  --tasks hellaswag \
  --num_fewshot 5 \
  --batch_size 5

# !time lm-eval --model vllm --model_args pretrained=TinyLlama/TinyLlama-1.1B-Chat-v1.0-W4A16,tokenizer=TinyLlama/TinyLlama-1.1B-Chat-v1.0,tensor_parallel_size=1 \
# --limit 250 --tasks hellaswag --num_fewshot 5 --batch_size 5

By comparsing the pre-quantized-eval and post-quantized-eval,

The value reduces from 0.572 to 0.564

Accuracy Recovery = Post-quantized-accuracy / Pre-quantized-accuracy.

So, the recovery here is 0.564 / 0.572 * 100% = 98.6%

Acutally, you have to run more than 500,000 evaluations on quantized models, which retains about 99% accuracy.